<a href="https://colab.research.google.com/github/sherif17/PySpark-For-Big-Data/blob/main/DF_SQL_ML_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Labs 1 and 2 PySpark:**

In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv file that we will be using in this lab is **PatientInfo**.

## PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

### Import the pyspark and check it's version

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

### Import and create SparkSession

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Load the PatientInfo.csv file and show the first 5 rows

In [ ]:
df=spark.read.csv('/content/PatientInfo.csv',header=True,inferSchema=True)
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

### Display the schema of the dataset

In [ ]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)



### Display the statistical summary

In [ ]:
df.describe().show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|          5162|         1587|           66|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9|1.6772572523506988E7|            

### Using the state column.
### How many people survived (released), and how many didn't survive (isolated/deceased)?

In [ ]:
from pyspark.sql.functions import count

df.groupBy("state").agg(count("state")).show()

+--------+------------+
|   state|count(state)|
+--------+------------+
|isolated|        2158|
|released|        2929|
|deceased|          78|
+--------+------------+



### Display the number of null values in each column

In [ ]:
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



## Data preprocessing

### Fill the nulls in the deceased_date with the released_date. 
- You can use <b>coalesce</b> function

In [ ]:
from pyspark.sql.functions import coalesce

df = df.withColumn("deceased_date", coalesce(df["deceased_date"], df["released_date"]))

### Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
- <b> Hint: You need to typecast these columns as date first <b>

In [ ]:
from pyspark.sql.functions import datediff, to_date
# Convert the confirmed_date and deceased_date columns to date type
df = df.withColumn("confirmed_date", to_date("confirmed_date", "yyyy-MM-dd"))
df = df.withColumn("deceased_date", to_date("deceased_date", "yyyy-MM-dd"))

# Calculate the difference between deceased_date and confirmed_date in days
df = df.withColumn("no_days", datediff(df["deceased_date"], df["confirmed_date"]))

# Show the top 5 rows and the schema
df.show(5)
df.printSchema()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|

### Add a is_male column if male then it should yield true, else then False

In [ ]:
# Import necessary modules
from pyspark.sql.functions import isnan

# Assume we have a DataFrame named `df` with a column named `sex`
# Use the `na` method to filter out rows with null or NaN values in `sex`
df = df.na.drop(subset=["sex"])

In [ ]:
from pyspark.sql.functions import when
df = df.withColumn("is_male", when(df["sex"] == "male", True).otherwise(False))

In [ ]:
# Show the top 5 rows and the schema
df.show(5)
df.printSchema()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|   true|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|   true|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact 

### Add a is_dead column if patient state is not released then it should yield true, else then False

- Use <b>UDF</b> to perform this task. 
- However, UDF is not recommended there is no built in function can do the required operation.
- UDF is slower than built in functions.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# Define a function to check if the patient is dead or not
def is_dead(state):
    return state != "released"

# Register the function as a UDF
is_dead_udf = udf(is_dead, BooleanType())

# Assume we have a DataFrame named `df` with a column named `state`
# Add an `is_dead` column using the UDF
df= df.withColumn("is_dead", is_dead_udf("state"))
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|   true|  false|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|   true|  false|
|1000000003|  male|5

### Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20

In [ ]:
from pyspark.sql.functions import regexp_extract, col

# Extract digits from ages column using regular expressions
df = df.withColumn("ages_extracted", regexp_extract(col("age"), "\\d{0,2}+", 0))

# Drop the intermediate columns
df = df.drop("age")

# Convert extracted digits to integer
df = df.withColumn("age", df["ages_extracted"].cast("int"))

# Drop the intermediate columns
df = df.drop("ages_extracted")

In [ ]:
df.show(5)

+----------+------+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+---+
|patient_id|   sex|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|age|
+----------+------+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+---+
|1000000001|  male|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|   true|  false| 50|
|1000000002|  male|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|   true|  false| 30|
|1000000003|  male| 

### Change age, and no_days  to be typecasted as Double

In [ ]:
from pyspark.sql.functions import col
df = df.withColumn("age", col("age").cast("double"))
df = df.withColumn("no_days", col("no_days").cast("double"))

### Drop the columns
["patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case"]

In [ ]:
columns_to_drop = ["patient_id", "sex", "infected_by", "contact_number", "released_date", "state", "symptom_onset_date", "confirmed_date", "deceased_date", "country", "no_days", "city", "infection_case"]
df = df.drop(*columns_to_drop)


In [ ]:
df.show(5)

+--------+-------+-------+----+
|province|is_male|is_dead| age|
+--------+-------+-------+----+
|   Seoul|   true|  false|50.0|
|   Seoul|   true|  false|30.0|
|   Seoul|   true|  false|50.0|
|   Seoul|   true|  false|20.0|
|   Seoul|  false|  false|20.0|
+--------+-------+-------+----+
only showing top 5 rows



### Recount the number of nulls now

In [ ]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+--------+-------+-------+---+
|province|is_male|is_dead|age|
+--------+-------+-------+---+
|       0|      0|      0|261|
+--------+-------+-------+---+



### Preparing For Modeling

In [ ]:
df.printSchema()

root
 |-- province: string (nullable = true)
 |-- is_male: boolean (nullable = false)
 |-- is_dead: boolean (nullable = true)
 |-- age: double (nullable = true)



In [ ]:
df


province,is_male,is_dead,age
Seoul,true,false,50.0
Seoul,true,false,30.0
Seoul,true,false,50.0
Seoul,true,false,20.0
Seoul,false,false,20.0
Seoul,false,false,50.0
Seoul,true,false,20.0
Seoul,true,false,20.0
Seoul,true,false,30.0
Seoul,false,false,60.0


In [ ]:
from pyspark.sql.functions import when
df = df.withColumn("is_dead", when(df.is_dead, 1).otherwise(0))


In [ ]:
df

province,is_male,is_dead,age
Seoul,true,0,50.0
Seoul,true,0,30.0
Seoul,true,0,50.0
Seoul,true,0,20.0
Seoul,false,0,20.0
Seoul,false,0,50.0
Seoul,true,0,20.0
Seoul,true,0,20.0
Seoul,true,0,30.0
Seoul,false,0,60.0


## Now do the same but using SQL select statement

### From the original Patient DataFrame, Create a temporary view (table).

In [ ]:
df_sql=spark.read.csv('/content/PatientInfo.csv',header=True,inferSchema=True)
df_sql.createOrReplaceTempView("patient_info")

### Use SELECT statement to select all columns from the dataframe and show the output.

In [ ]:
# select all columns from the dataframe using SELECT statement
query = '''
        SELECT *
        FROM patient_info
        '''

# execute the query and display the output
result = spark.sql(query)
result.show()


+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

### *Using SQL commands*, limit the output to only 5 rows 

In [ ]:
spark.sql("SELECT * FROM patient_info LIMIT 5").show()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

### Select the count of males and females in the dataset

In [ ]:
spark.sql("SELECT sex, COUNT(*) AS count FROM patient_info GROUP BY sex")

sex,count
null,1122
female,2218
male,1825


### How many people did survive, and how many didn't?

In [ ]:
spark.sql("SELECT state, count(*) AS count FROM patient_info GROUP BY state")

state,count
isolated,2158
released,2929
deceased,78


### Now, let's perform some preprocessing using SQL:
1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [ ]:
result_df = spark.sql("SELECT  sex, CAST(SUBSTRING(age, 1, LENGTH(age) - 1) AS double) as age, province, state FROM patient_info")
result_df.show(5)

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
+------+----+--------+--------+
only showing top 5 rows



## Machine Learning 
### Create a pipeline model to predict is_dead and evaluate the performance.
- Use <b>StringIndexer</b> to transform <b>string</b> data type to indices.
- Use <b>OneHotEncoder</b> to deal with categorical values.
- Use <b>Imputer</b> to fill missing data with mean.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Imputer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


# Split the data into train and test sets
trainData, testData = df.randomSplit([0.7, 0.3], seed=42)

# Define the stages for the pipeline
stringIndexer = StringIndexer(inputCols=["province"], outputCols=["provinceIndex"])

oneHotEncoder = OneHotEncoder(inputCols=["provinceIndex"], outputCols=["provinceVec"])

imputer = Imputer(inputCols=["age"], outputCols=["age_imputed"], strategy="mean")

assembler = VectorAssembler(inputCols=["provinceVec", "is_male", "age_imputed"], outputCol="features")

logReg = LogisticRegression(featuresCol="features", labelCol="is_dead")

# Define the pipeline
pipeline = Pipeline(stages=[stringIndexer, oneHotEncoder, imputer, assembler, logReg])

# Fit the pipeline to the training data
model = pipeline.fit(trainData)

# Make predictions on the training data
train_predictions = model.transform(trainData)

# Evaluate the model on the training data
train_evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="is_dead")
train_auc = train_evaluator.evaluate(train_predictions)

# Make predictions on the test data
test_predictions = model.transform(testData)

# Evaluate the model on the test data
test_evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="is_dead")
test_auc = test_evaluator.evaluate(test_predictions)

print("Training AUC:", train_auc)
print("Test AUC:", test_auc)


Training AUC: 0.9229486053283915
Test AUC: 0.9301400403166592
